In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def author_ballot_dictionary(Weeks, url, year):
    Dictionaries = []
    for week in Weeks:
        new_url = url + week
        response = requests.get(new_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
        else:
            print("bad")
            continue
        parent_element = soup.find('div', id='gridBallots')
        Dict = {}
        if parent_element:
            Rows = parent_element.find_all('div', class_='gridRow')
            for row in Rows:
                author_name = row.find('div', class_='gridPollster').get_text(strip=True)
                Team_rows = row.find_all('div', class_= 'gridTeam')
                team_names = [author_name, year, Weeks.index(week) + 1]
                for item in Team_rows:
                    image = item.find('img')
                    if not image:
                        continue
                    team_name = image.get('title')
                    team_names.append(team_name)
                    Dict[author_name] = team_names
        Dictionaries.append(Dict)

    return Dictionaries

def csv_data_appender_by_year(year): 
    Weeks = ['pre-season', 'week-2', 'week-3', 'week-4', 'week-5', 'week-6', 'week-7', 'week-8', 'week-9', 'week-10', 'week-11', 'week-12', 'week-13', 'week-14', 'week-15', 'week-16', 'final-rankings']
    url = "https://collegepolltracker.com/football/grid/" + year + "/"
    Dictionaries = author_ballot_dictionary(Weeks, url, year)

    for dicto in Dictionaries:
        df = pd.DataFrame.from_dict(dicto, orient='index', columns=['Pollster', 'Season', 'Week', '1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th', '11th', '12th', '13th', '14th', '15th', '16th', '17th', '18th', '19th', '20th', '21st', '22nd', '23rd', '24th', '25th'])
        df.to_csv('data.csv', mode='a', index=False, header=False)


def csv_creation():
    years = ["2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023", "2024"]
    
    for year in years:
        csv_data_appender_by_year(year)

csv_creation()